In [2]:
import pymysql as SQL
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [3]:
Host = "82.148.19.206"
User = "molozey"
Password = "Exeshnik08!"
DataBaseName = "FinTechData"

In [4]:
db = SQL.connect(host=Host, user=User, password=Password, database=DataBaseName)
cursor = db.cursor()

INFO = cursor.execute("""SELECT Indicators.*, Sector.Sector, Info.Country
                            FROM Ticker_Information as Info
                            JOIN Ticker_Sector as Sector USING (Ticker)
                            JOIN Ticker_Indicators as Indicators USING (Ticker)""")
INFO = cursor.fetchall()
INFO_COLUMNS = [_[0] for _ in cursor.description]
db.close()
DF_INFO = pd.DataFrame(INFO, columns=INFO_COLUMNS)
DF_INFO

,Ticker,Update_Date,Last_Price,Market_Cap,Earnings,Revenue_Growth,Price_To_Earnings,Sector,Country
0,AACG,2021-12-10 06:30:53,1.57,4.977090e+07,-9.696990e+07,0.149,NaN,Miscellaneous,China
1,AACI,2021-12-10 06:30:53,NaN,NaN,NaN,NaN,NaN,Not Available...,United States
2,AACIU,2021-12-10 06:30:53,NaN,NaN,NaN,NaN,NaN,Not Available...,United States
3,AACIW,2021-12-10 06:30:53,NaN,NaN,NaN,NaN,NaN,Not Available...,United States
4,AADI,2021-12-10 06:30:53,22.40,NaN,NaN,NaN,NaN,Health Care,United States
...,...,...,...,...,...,...,...,...,...
70,ADBE,2021-12-10 06:30:53,632.57,3.023680e+11,5.582000e+09,0.226,54.8249,Technology,United States
71,ADER,2021-12-10 06:30:53,9.89,3.399690e+08,-3.054000e+03,NaN,NaN,Not Available...,United States
72,ADERU,2021-12-10 06:30:53,10.32,NaN,-3.054000e+03,NaN,NaN,Not Available...,United States
73,ADERW,2021-12-10 06:30:53,NaN,NaN,NaN,NaN,NaN,Not Available...,United States


In [5]:
#   SIMILAR NEIGHBORS

DF_INFO = pd.DataFrame(INFO, columns=INFO_COLUMNS)
scaler = MinMaxScaler()
DF_INFO[['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings']] = scaler.fit_transform(DF_INFO[['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings']])
encode = LabelEncoder()
DF_INFO['Country'] = encode.fit_transform(DF_INFO['Country'])
DF_INFO['Sector'] = encode.fit_transform(DF_INFO['Sector'])

weights = [0.05, 0.05, 0.05, 0.1, 0.15, 0.3, 0.2]
#neighbors = NearestNeighbors(n_neighbors=5, radius=1, metric_params={})
neighbors = NearestNeighbors(algorithm='brute',
                        metric='wminkowski',
                        metric_params={'w': weights},
                        p=1,
                        n_jobs=1,
                        n_neighbors=5)
neighbors.fit(DF_INFO.replace(np.NAN, -1)[['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings', 'Sector', 'Country']].values)
TICKERS_LABELS = DF_INFO.Ticker
obj = DF_INFO.iloc[0].replace(np.NAN, -1)[['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings', 'Sector', 'Country']].values
print(obj)
dist, labels = neighbors.kneighbors([obj], n_neighbors=5)

DF_INFO.iloc[labels[0]]

[0.0015661645172121447 6.81310402006617e-06 0.04607668975686084
 0.37574404761904767 -1 5 2]


/Users/molozey/Desktop/FinTechProject_old/venv/lib/python3.9/site-packages/sklearn/base.py:561: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


,Ticker,Update_Date,Last_Price,Market_Cap,Earnings,Revenue_Growth,Price_To_Earnings,Sector,Country
0,AACG,2021-12-10 06:30:53,0.001566,0.000007,0.046077,0.375744,NaN,5,2
67,ADAG,2021-12-10 06:30:53,0.014383,0.000134,0.046673,0.251488,NaN,4,2
31,ACBA,2021-12-10 06:30:53,0.015016,NaN,0.047140,NaN,NaN,5,3
32,ACBAW,2021-12-10 06:30:53,NaN,NaN,NaN,NaN,NaN,5,3
22,ABST,2021-12-10 06:30:53,0.013323,0.000143,0.047182,0.383557,NaN,6,1


In [6]:
DF_INFO[['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings']]

,Last_Price,Market_Cap,Earnings,Revenue_Growth,Price_To_Earnings
0,0.001566,0.000007,0.046077,0.375744,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,0.034526,NaN,NaN,NaN,NaN
...,...,...,...,...,...
70,1.000000,0.103791,0.108417,0.404390,0.731244
71,0.014731,0.000106,0.047141,NaN,NaN
72,0.015411,NaN,0.047141,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN


In [7]:
#   FIFA PREPROCESSING OLD VERSION USING PANDAS
"""
values = DF_INFO.replace(np.NAN, 0)['Last_Price'].values
def fifa_Scaler(values):
    OldMax = max(values)
    OldMin = min(values)

    NewMax= 100
    NewMin = 30

    OldRange = (OldMax - OldMin)
    NewRange = (NewMax - NewMin)
    NewValues = [int((((OldValue - OldMin) * NewRange) // OldRange) + NewMin) for OldValue in values]
    return NewValues

def FIFA(df: pd.DataFrame, columns):
    df.replace(np.NAN, 0, inplace=True)
    for column in columns:
        df[column] = fifa_Scaler(df[column].values)
    return df

FIFA(DF_INFO, columns=['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth', 'Price_To_Earnings'])
"""

"\nvalues = DF_INFO.replace(np.NAN, 0)['Last_Price'].values\ndef fifa_Scaler(values):\n    OldMax = max(values)\n    OldMin = min(values)\n\n    NewMax= 100\n    NewMin = 30\n\n    OldRange = (OldMax - OldMin)\n    NewRange = (NewMax - NewMin)\n    NewValues = [int((((OldValue - OldMin) * NewRange) // OldRange) + NewMin) for OldValue in values]\n    return NewValues\n\ndef FIFA(df: pd.DataFrame, columns):\n    df.replace(np.NAN, 0, inplace=True)\n    for column in columns:\n        df[column] = fifa_Scaler(df[column].values)\n    return df\n\nFIFA(DF_INFO, columns=['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth', 'Price_To_Earnings'])\n"

In [8]:
#   FIFA PREPROCESSING OLD VERSION USING SQL
def FIFA(ticker_name:str, columns:list):
    """

    :param ticker_name: Ticker name which data need to be converted
    :param columns: Columns that may be invert to FIFA Scale
    :return: JSON with ticker and COLUMNS
    """

    def fifa_Scaler(value, OldMax:float, OldMin:float):
        """

        :param value: value to convert
        :param OldMax: right border (MAX RATE)
        :param OldMin: left border (MIN RATE)
        :return: Beautiful rescaled value + total_rating
        """
        NewMax = 100 #   MAX VALUE
        NewMin = 30 #   MIN VALUE. BE CAREFUL NONE VALUES == NEWMIN

        OldRange = (OldMax - OldMin)
        NewRange = (NewMax - NewMin)
        if value:
            NewValues = int((((value - OldMin) * NewRange) // OldRange) + NewMin)
        else:
            NewValues = NewMin
        return NewValues

    ret_dict = dict()
    ret_dict.update({'ticker': ticker_name})
    for COL in columns:
        db = SQL.connect(host=Host, user=User, password=Password, database=DataBaseName)
        cursor = db.cursor()

        F_INFO = cursor.execute(f"""SELECT MAX(Indicators.{COL}), MIN(Indicators.{COL})
                                    FROM Ticker_Information as Info
                                    JOIN Ticker_Sector as Sector USING (Ticker)
                                    JOIN Ticker_Indicators as Indicators USING (Ticker)""")
        F_INFO = cursor.fetchall()

        ticker_pos = cursor.execute(f"""SELECT Indicators.{COL}
                                        FROM Ticker_Indicators as Indicators WHERE Indicators.Ticker=%s""", ticker_name)
        ticker_pos = cursor.fetchone()[0]
        db.close()

        (ColMax, ColMin,) = F_INFO[0]
        new_value = fifa_Scaler(ticker_pos, OldMax=ColMax, OldMin=ColMin)
        ret_dict.update({COL: new_value})
        SUMMARY = 0
        counter = 0
        for _ in ret_dict.items():
            if _[0] != "ticker":
                counter += 1
                SUMMARY += ret_dict[_[0]]
        ret_dict.update({'total_rating': round(SUMMARY / counter)})

    return ret_dict

FIFA_COLUMNS = ['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings']
da = FIFA('ADES', FIFA_COLUMNS)
da

{'ticker': 'ADES',
 'Last_Price': 30,
 'total_rating': 39,
 'Market_Cap': 30,
 'Earnings': 33,
 'Revenue_Growth': 70,
 'Price_To_Earnings': 30}

In [9]:
DF_INFO[DF_INFO.Ticker == 'AACG']

,Ticker,Update_Date,Last_Price,Market_Cap,Earnings,Revenue_Growth,Price_To_Earnings,Sector,Country
0,AACG,2021-12-10 06:30:53,0.001566,0.000007,0.046077,0.375744,NaN,5,2


In [10]:
def get_similar(ticker, Host, User, Password, DataBaseName):
    """

    :param ticker: Name ticker to find similar tickers
    :param Host: DataBase Host
    :param User: DataBase User
    :param Password: DataBase Password
    :param DataBaseName: DataBase Name
    :return: list of similar tickers
    """
    db = SQL.connect(host=Host, user=User, password=Password, database=DataBaseName)
    cursor = db.cursor()

    INFO = cursor.execute("""SELECT Indicators.*, Sector.Sector, Info.Country
                                FROM Ticker_Information as Info
                                JOIN Ticker_Sector as Sector USING (Ticker)
                                JOIN Ticker_Indicators as Indicators USING (Ticker)""")
    INFO = cursor.fetchall()
    INFO_COLUMNS = [_[0] for _ in cursor.description]
    db.close()

    #   SIMILAR NEIGHBORS

    DF_INFO = pd.DataFrame(INFO, columns=INFO_COLUMNS)
    scaler = MinMaxScaler()
    DF_INFO[['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings']] = scaler.fit_transform(DF_INFO[['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings']])
    encode = LabelEncoder()
    DF_INFO['Country'] = encode.fit_transform(DF_INFO['Country'])
    DF_INFO['Sector'] = encode.fit_transform(DF_INFO['Sector'])

    weights = [0.05, 0.05, 0.05, 0.1, 0.15, 0.3, 0.2]
    #neighbors = NearestNeighbors(n_neighbors=5, radius=1, metric_params={})
    neighbors = NearestNeighbors(algorithm='brute',
                            metric='wminkowski',
                            metric_params={'w': weights},
                            p=1,
                            n_jobs=1,
                            n_neighbors=5)
    neighbors.fit(DF_INFO.replace(np.NAN, -1)[['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings', 'Sector', 'Country']].values)
    TICKERS_LABELS = DF_INFO.Ticker
    obj = DF_INFO[DF_INFO.Ticker == ticker].replace(np.NAN, -1)[['Last_Price', 'Market_Cap', 'Earnings', 'Revenue_Growth','Price_To_Earnings', 'Sector', 'Country']].values[0]
    dist, labels = neighbors.kneighbors([obj], n_neighbors=5)
    DF_INFO = DF_INFO.iloc[labels[0]]
    return DF_INFO.iloc[1:].Ticker.values


#[0.0015661645172121447 6.81310402006617e-06 0.04607668975686084 0.37574404761904767 -1 5 2]
get_similar('AACG', Host=Host, Password=Password, User=User, DataBaseName=DataBaseName)

array(['ADAG', 'ACBA', 'ACBAW', 'ABST'], dtype=object)